In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [2]:
price_data = pd.read_csv("price.csv")

C:\Users\Shawn Z\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,6,9,19,48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
PERMNO = set(price_data["PERMNO"].values)

In [4]:
len(PERMNO)

570

In [5]:
def produce_split_rate(price_data, stock_code_list):
    price_data_dict = {}
    
    for i in stock_code_list:
        one_stock_data = price_data.loc[price_data["PERMNO"]==i]
        one_stock_data["split"] = (one_stock_data["CFACSHR"]/one_stock_data["CFACSHR"].shift(1)).fillna(1)
        one_stock_data["DIVAMT"] = one_stock_data["DIVAMT"].fillna(0)
        one_stock_data = one_stock_data.sort_values(by = ["date"]).reset_index(drop = True)
        price_data_dict[str(i)] = one_stock_data[["PERMNO", "date", "TICKER", "DIVAMT", "PRC", "VOL",
                                                  "OPENPRC", "split", "NAICS"]]
    return price_data_dict

In [6]:
price_data_dict = produce_split_rate(price_data, PERMNO)

C:\Users\Shawn Z\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Shawn Z\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
def get_adj_price(raw_data, start_date):
    
    raw_data = raw_data[raw_data['date'] >= start_date]
    
    raw_data['CSP'] = raw_data['split'].cumprod()
    
    raw_data['adj_DIVAMT'] = raw_data['DIVAMT']*raw_data['CSP']
    
    raw_data['adj_DIVAMT'] = raw_data['adj_DIVAMT'].cumsum()
    
    raw_data['adj_open'] = raw_data['OPENPRC']*raw_data['CSP'] + raw_data['adj_DIVAMT']
    
    raw_data['adj_close'] = raw_data['PRC']*raw_data['CSP'] + raw_data['adj_DIVAMT']
    
    raw_data['adj_volume'] = raw_data['VOL']/raw_data['CSP']
    
    index_missing = np.where(raw_data['adj_volume'] == 0)[0]
    
    raw_data['adj_volume'] = raw_data['adj_volume'].replace(to_replace = 0, method = 'ffill')
    
    raw_data['adj_close'] = raw_data['adj_close'].fillna(raw_data['adj_open'])
    
    raw_data.loc[raw_data['adj_close'] > 1000,'adj_close'] = raw_data.loc[raw_data['adj_close'] > 1000,'adj_open']
    
    raw_data['date'] = pd.to_datetime(raw_data['date'].astype(str))
    
    return raw_data[['date',"PERMNO","TICKER",'adj_open','adj_close','adj_volume',"NAICS"]].set_index('date')

In [8]:
new_data = get_adj_price(price_data_dict["34817"], 20050103)

## Specify the periods
1. Breakdown 2005 - 2012 into 4 periods for factor analysis 
    - Each period is 2 years
    - Rolling regression, fitting window = 1.5year
    -
2. Rest of data is for Backtesting

## Signgle Fcator Analysis
- volume

In [3]:
data = pd.read_parquet('final_data.pq',engine = 'pyarrow')

In [4]:
from BTC_Alpha_func import *
x = data['adj_volume'].groupby('stock').apply(lambda x: x.rolling(20).apply(lambda x:x[-1]/x.sum()))

C:\Users\Shawn Z\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  


In [20]:
def vol_adv20(data):
    return data[['adj_volume']].groupby('stock').apply(lambda x:x/SUM(x, 20))

def vol_open_close(data):
    fac = pd.DataFrame(data['adj_volume'] / (data['adj_open'] - data['adj_close']))
    return fac.groupby('stock').apply(lambda x:EWM(x/SUM(x,20),20))

def prc_momentum(data):
    # 20-day-return
    fac = data.groupby('stock')['adj_close'].apply(lambda x: x/DELAY(x, 20))
    return fac

def abs_ret_sum(data):
    # 20-abs-return-sum
    ret = data.groupby('stock')['adj_close'].apply(lambda x:x/DELAY(x, 1)-1)
    fac = ret.groupby('stock').apply(lambda x: SUM(ABS(x),20))
    return fac   

def neg_close(data):
    return -data['adj_close']




def short_debt_equity(data):
    #3051/3501
    fac = pd.DataFrame(data['ITEM3051'] / data['ITEM3501'])
    return fac

def sales_asset(data):
    #18198/2201
    #EBITDA/ASSET
    fac = pd.DataFrame(data['ITEM18198'] / data['ITEM2201'])
    return fac

neg_close(data)
#prc_reversion(data)


stock  date      
10104  2005-01-03    -13.410000
       2005-01-04    -13.060000
       2005-01-05    -13.100000
       2005-01-06    -13.220000
       2005-01-07    -13.330000
       2005-01-10    -13.190000
       2005-01-11    -13.200000
       2005-01-12    -13.480000
       2005-01-13    -13.480000
       2005-01-14    -13.630000
       2005-01-18    -13.780000
       2005-01-19    -13.470000
       2005-01-20    -13.280000
       2005-01-21    -13.310000
       2005-01-24    -13.240000
       2005-01-25    -13.590000
       2005-01-26    -13.620000
       2005-01-27    -13.970000
       2005-01-28    -13.680000
       2005-01-31    -13.770000
       2005-02-01    -13.650000
       2005-02-02    -13.560000
       2005-02-03    -13.340000
       2005-02-04    -13.660000
       2005-02-07    -13.550000
       2005-02-08    -13.470000
       2005-02-09    -13.170000
       2005-02-10    -13.140000
       2005-02-11    -13.350000
       2005-02-14    -13.310000
                      

In [57]:
data['']

ITEM1001     ITEM1051      ITEM1100      ITEM1101  \
stock date                                                                
10104 2005-01-03  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-04  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-05  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-06  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-07  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   

                     ITEM1151  ITEM1230      ITEM1250    ITEM1251  ITEM1253  \
stock date                                                                    
10104 2005-01-03  186000000.0       0.0  1.552000e+09  66000000.0       0.0   
      2005-01-04  186000000.0       0.0  1.552000e+09  66000000.0       0.0   
      2005-01-05  186000000.0       0.0  1.552000e+09  66000000.0       0.0   
      2005-01-06  186000000.0       0.0  1.552000e+09  66000000.0       0.0   
      2005-01-07  186000000.0       0.0  1.552000e+09  66000000.0       0.0   

                     ITEM1254     ...      ITEM8621  ITEM8631  ITEM8636  \
stock date                        ...                                     
10104 2005-01-03  145000000.0     ...      63.01002  16.17763    7.6464   
      2005-01-04  145000000.0     ...      63.01002  16.17763    7.6464   
      2005-01-05  145000000.0     ...      63.01002  16.17763    7.6464   
      2005-01-06  145000000.0     ...      63.01002  16.17763    7.6464   
      2005-01-07  145000000.0     ...      63.01002  16.17763    7.6464   

                  ITEM8651  ITEM8676  ITEM8698  ITEM8699  adj_close  adj_open  \
stock date                                                                      
10104 2005-01-03  30.11685  -7.34327  26.07282   5.26316      13.41     13.86   
      2005-01-04  30.11685  -7.34327  26.07282   5.26316      13.06     13.46   
      2005-01-05  30.11685  -7.34327  26.07282   5.26316      13.10     13.03   
      2005-01-06  30.11685  -7.34327  26.07282   5.26316      13.22     13.15   
      2005-01-07  30.11685  -7.34327  26.07282   5.26316      13.33     13.34   

                  adj_volume  
stock date                    
10104 2005-01-03  60640045.0  
      2005-01-04  81022553.0  
      2005-01-05  43102873.0  
      2005-01-06  55964351.0  
      2005-01-07  45705174.0  

[5 rows x 130 columns]

In [64]:
x = data[['adj_close']].groupby('stock').apply(lambda x:x/DELAY(x,1)-1)
x[abs(x)>.5].dropna()

adj_close
stock date                 
10138 2006-06-26  -0.735169
10696 2013-12-17  -0.750353
      2018-03-20  -0.750543
10909 2006-01-10  -0.753501
      2011-06-27  -0.745863
      2013-07-01  -0.746332
11308 2012-08-13  -0.654711
11552 2006-02-27  -0.740132
      2014-06-26  -0.749537
11600 2006-07-18  -0.742311
11618 2005-11-14  -0.742854
      2011-05-23  -0.694390
11762 2011-03-01  -0.688227
11891 2005-05-19  -0.743697
12052 2006-03-27  -0.741950
12073 2005-05-16  -0.544045
      2007-04-16  -0.728065
12431 2005-09-02  -0.742081
12570 2006-02-22  -0.745393
      2011-11-02   1.861098
13928 2005-06-02  -0.743102
14277 2006-04-10  -0.743200
14461 2005-01-04  -1.987256
      2005-01-05  -2.037206
      2005-01-07  -2.016231
      2005-01-10  -2.003534
      2005-01-11  -2.003521
      2005-01-12  -2.080702
      2005-01-18   1.312500
      2005-05-25  -1.913357
...                     ...
88446 2008-09-23  -0.754939
88661 2005-04-05  -0.744448
88837 2006-08-16  -0.736819
88845 2005-03-14  -0.746937
      2006-02-21  -0.754221
88853 2008-07-17  -0.734990
88873 2005-10-31  -0.749389
      2015-11-03  -0.746339
89004 2007-09-04  -0.738807
      2011-04-01  -0.719582
89179 2005-06-01  -0.749586
89217 2005-09-26  -0.561921
89269 2015-02-20  -0.752010
89393 2015-07-15  -0.980048
89508 2007-03-27  -0.745209
89525 2007-02-22  -0.561046
      2017-02-21  -0.694915
89626 2012-07-23  -0.831848
89866 2006-01-17  -0.752835
      2007-12-04  -0.749622
90071 2007-06-01  -0.747241
90199 2006-06-02  -0.885909
      2009-03-25   0.640000
90215 2013-04-18  -0.938706
90272 2007-10-25  -0.629504
      2008-11-13  -0.542023
91416 2015-03-04  -0.921014
91556 2011-12-16  -0.722761
      2015-06-12  -0.690709
92655 2005-05-31  -0.751329

[1556 rows x 1 columns]

In [33]:
def vol_fac(stock_price_table):
    fac = stock_price_table['adj_volume'].rolling(20).apply(lambda x: x[-1]/x.sum())
    return fac

vol_fac(data).tail()

C:\Users\Shawn Z\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  


stock  date      
92655  2018-06-25    0.058472
       2018-06-26    0.041768
       2018-06-27    0.047642
       2018-06-28    0.067537
       2018-06-29    0.052518
Name: adj_volume, dtype: float64

In [23]:
data

ITEM1001     ITEM1051      ITEM1100      ITEM1101  \
stock date                                                                
10104 2005-01-03  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-04  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-05  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-06  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-07  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-10  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-11  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-12  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-13  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-14  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-18  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-19  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-20  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-21  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-24  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-25  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-26  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-27  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-28  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-01-31  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-01  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-02  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-03  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-04  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-07  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-08  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-09  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-10  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-11  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
      2005-02-14  3.878000e+09  726000000.0  2.966000e+09  1.414000e+09   
...                        ...          ...           ...           ...   
92655 2018-05-18  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-21  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-22  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-23  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-24  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-25  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-29  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-30  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-05-31  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-01  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-04  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-05  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-06  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-07  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-08  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-11  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-12  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-13  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-14  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-15  4.872300e+10  572542000.0  7.248000e+09  1.323100e+10   
      2018-06-18  4.872300e+10  572542000.0  7.

In [21]:
def vol_close_open(stock_price_table):
    def temp(to_roll):
        return to_roll['adj_volume']/(to_roll['adj_close'] - to_roll['adj_open'])
    
    fac = stock_price_table.rolling(20).apply(lambda x: temp(x)/temp(x).sum())
    
    
    return fac

vol_close_open(data).tail()

C:\Users\Shawn Z\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices